## Lab 5: Supervised classification and regression

**Purpose:**
The purpose of this lab is introduce you to concepts of supervised classification and regression: prediction of nominal or numeric values of a geographic variable from other geographic variables.  You will explore processes of training data collection, classifier selection, classifier training, image classification and accuracy assessment.  At the completion of the lab, you will be able to perform supervised classification and regression in Earth Engine.

**Prerequisites:** Lab 4

### 1. Introduction to classification and regression
For present purposes, define prediction as guessing the value of some geographic variable of interest *g*, using a function *G* that takes as input a pixel vector **p**:

\begin{equation}
G_{T}(p_i) = g_i 
\end{equation}

The *i* in this equation refers to a particular instance from a set of pixels.  Think of *G* as a guessing function and *gi* as the guess for pixel *i*.   The **T** in the subscript of *G* refers to a *training set* (a set of known values for p and the correct g), used to infer the structure of G.  You have to choose a suitable *G* to train with **T**.  When *g* is nominal (e.g. {'water', 'vegetation', 'bare'}), call this setup classification.  When g is numeric, call this setup regression.  This is an incredibly simplistic description of a problem addressed in a broad range of fields including mathematics, statistics, data mining, machine learning, etc.  Interested readers may see [Witten et al. (2011)](http://www.cs.waikato.ac.nz/ml/weka/book.html), [Hastie et al. (2009)](http://statweb.stanford.edu/~tibs/ElemStatLearn/) or [Goodfellow et al (2016)](http://www.deeplearningbook.org/).

###  2. Regression
In the present context, regression means predicting a numeric variable instead of a class label.  No lab on regression would be complete without the requisite introduction to least squares regression.

#### a. Ordinary Least Squares (OLS)

A very [ordinary regression](https://en.wikipedia.org/wiki/Ordinary_least_squares) is when G is a linear function of the form G(**p**) = **βp** where **β** is a vector of coefficients.  Once G is trained by some training set **T**, guess the value for some unknown **p** by multiplying it with **β**.  Suppose the goal is to estimate percent tree cover in each Landsat pixel.

- Import data to use as known values for g.  Search 'vegetation continuous fields' and import 'MOD44B'.  Get the most recent image out of this collection:

In [131]:
# Initializing display and earthengine
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

# importing ipygee for dynamic mapping
from ipygee import *

Map = Map() # from ipygee
Map.show()

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

In [132]:
mod44b =ee.ImageCollection("MODIS/006/MOD44B")

tree = ee.Image(mod44b.sort('system:time_start', False).first())

Since water is coded as 200 in this image, replace the 200's with 0's and display the result:

In [133]:
percentTree = tree.select('Percent_Tree_Cover').where(tree.select('Percent_Tree_Cover').eq(200), 0)

Map.addLayer(percentTree, {'max': 100, 'palette':['red','yellow', 'green', 'blue', 'purple']}, 
             name='percent tree cover')

We could benefit from a stronger or more detailed water occurrence such as *JRC/GSW1_0/GlobalSurfaceWater*. Let's create a water mask for it.

In [134]:
waterOcc = ee.Image("JRC/GSW1_0/GlobalSurfaceWater").select('occurrence')
jrc_data0 = ee.Image("JRC/GSW1_0/Metadata").select('total_obs').lte(0)
waterOccFilled = waterOcc.unmask(0).max(jrc_data0)
waterMask = waterOccFilled.lt(50)

Map.addLayer(waterMask,[],'WaterMask')

Let's update the *percentTree* layer with the *waterMask* and plot it.

In [135]:
percentTreeWM = percentTree.updateMask(waterMask)

Map.addLayer(percentTreeWM, {'max': 100, 'palette':['red','yellow', 'green', 'blue', 'purple']},
             name='percent tree cover WM')

Note that this image represents percent tree cover at 250 meter resolution in 2010.

- Import data to use as predictor variables (**p**).  Search 'landsat 5 raw' and import 'USGS Landsat 5 TM Collection 1 Tier 1 Raw Scenes'.  Name the import l5raw.  Filter by time and the [WRS-2](http://landsat.usgs.gov/worldwide_reference_system_WRS.php) path and row to get only scenes over the San Francisco bay area in 2010:


In [136]:
l5raw = ee.ImageCollection("LANDSAT/LT05/C01/T1")

l5filtered = l5raw.filterDate('2010-01-01', '2010-12-31').\
filterMetadata('WRS_PATH', 'equals', 44).\
filterMetadata('WRS_ROW', 'equals', 34)

Use an Earth Engine algorithm to get a cloud-free composite of Landsat imagery in 2010. Also apply the *waterMask* mask to the *landsat* composite.

In [137]:
landsat = ee.Algorithms.Landsat.simpleComposite(l5filtered,50,10,40, True)

landsat = landsat.updateMask(waterMask)
# print(landsat.getInfo())
Map.addLayer(landsat, {'bands': ['B4', 'B3', 'B2'], 'max': 0.3}, 'composite')

Map.centerObject(l5filtered.first().geometry()) #centering the map to landsat scene

Specify the bands of the Landsat composite to be used as predictors (i.e. the elements of p):


In [138]:
predictionBands = ['B1', 'B2', 'B3', 'B4', 'B5' ,'B7']

Now that all the input data is ready, build T.  It's customary to include a constant term in linear regression to make it the best linear unbiased estimator.  Stack a constant, the predictor variables and the image representing known g:


In [139]:
trainingImage = ee.Image(1).addBands(landsat.select(predictionBands)).addBands(percentTreeWM)

Sample this stack at 1000 locations to get T as a table:


In [140]:
region = l5filtered.first().geometry()

# // Create 1000 random points in the region.
randomPoints = ee.FeatureCollection.randomPoints(region);

# // Display the points.
Map.centerObject(randomPoints);
Map.addLayer(randomPoints, {}, 'random points')

training = trainingImage.sampleRegions(
  collection= randomPoints, 
  scale= 30, 
)

print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'B1': 0.11413625627756119, 'B2': 0.11237142235040665, 'B3': 0.11597663164138794, 'B4': 0.23223663866519928, 'B5': 0.21078398823738098, 'B7': 0.144679456949234, 'Percent_Tree_Cover': 15, 'constant': 1}}


Inspect the first element of T to make sure it's got all the data you expect.  

The next step is to train G.  Make a list of the variable names, predictors followed by g:


In [141]:
trainingList = ee.List(predictionBands).insert(0, 'constant').add('Percent_Tree_Cover')
print(trainingList.getInfo())

['constant', 'B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'Percent_Tree_Cover']


In Earth Engine, linear regression is implemented as a Reducer.  This means that training G is a reduction of the T table, performed using the list of variables as an input.  The argument (8) tells the reducer how many of the input variables are predictors:

In [142]:
regression = training.reduceColumns(
  reducer= ee.Reducer.linearRegression(7), 
  selectors= trainingList
)

Observe that the output is an object with a list of coefficients (in the order specified by the inputs list) and the root mean square residual.  To use the coefficients to make a prediction in every pixel, first turn the output coefficients into an image, then perform the multiplication and addition that implements βp:


In [143]:
coefficients = ee.Array(regression.get('coefficients')).project([0]).toList()
print(coefficients.getInfo())

[64.60438265752828, -19.590167766701438, -569.3868333268703, 281.6166852657393, 65.2857489660617, -222.59630246097112, 92.30175489558329]


In [144]:
predictedTreeCover = ee.Image(1).addBands(landsat.select(predictionBands)).\
multiply(ee.Image.constant(coefficients)).reduce(ee.Reducer.sum()).rename('predictedTreeCover')

# clipping image to Landsat footprint
predictedTreeCover = predictedTreeCover.clip(l5filtered.first().geometry())
# applying water mask
predictedTreeCover = predictedTreeCover.updateMask(waterMask)

Map.addLayer(predictedTreeCover, {'min': 0, 'max': 100,
                                  'palette':['red','yellow', 'green', 'blue', 'purple']}, 'OLS prediction')

### - Non-linear regression functions

If the garden variety linear regression isn't satisfactory, Earth Engine contains other functions that can make predictions of a continuous variable.  Unlike linear regression, other regression functions are implemented by the classifier library.  

- For example, a Classification and Regression Tree (CART, see Brieman et al. 1984) is a machine learning algorithm that can learn non-linear patterns in your data.  Reusing the T table (without the constant term), train a CART as follows:

In [145]:
cartRegression = ee.Classifier.cart().setOutputMode('REGRESSION').train(
      features= training, 
      classProperty= 'Percent_Tree_Cover', 
      inputProperties= predictionBands
    );

- Make predictions over the input imagery (classify in this context is a misnomer):


In [146]:
cartRegressionImage = landsat.select(predictionBands).classify(cartRegression, 'cartRegression');

Map.addLayer(cartRegressionImage, {'min': 0, 'max': 100,
                                   'palette':['red','yellow', 'green', 'blue', 'purple']},
             'CART regression');

Let's compare the performance in a numerical manner

In [147]:
olsresults= predictedTreeCover.sampleRegions(collection=randomPoints, scale=30)
print(olsresults.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'predictedTreeCover': 12.642754758141258}}


In [148]:
cartresults= cartRegressionImage.sampleRegions(collection=randomPoints, scale=30)
print(cartresults.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '0_0', 'properties': {'cartRegression': 13.149443626403809}}


In [168]:
cartresults= percentTreeWM.sampleRegions(collection=randomPoints, scale=30)
print(cartresults.propertyNames().getInfo())

['band_order']


In [167]:
aa=cartresults.getArray('Percent_Tree_Cover')
print(aa.getInfo())

EEException: Array: Parameter 'values' is required.

In [162]:
import pandas as pd
df =pd.DataFrame(list(cartresults.getArray()),columns=['Percent_Tree_Cover'])
# # sort bands
# df.sort_values('Band', inplace=True) 
# # add wavelenth data to frame
# df = df.assign(Wavelength=wavelengths)
df

EEException: Required argument (property) missing to function: Extract a property from a feature.

Args:
  object: The feature to extract the property from.
  property: The property to extract.